<a href="https://colab.research.google.com/github/lherron2/thermomaps-ising/blob/main/Ising_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone repo
%cd /content/
!rm -rf thermomaps-ising
!git clone https://github.com/lherron2/thermomaps-ising.git

# Install repo
%cd /content/thermomaps-ising/thermomaps-root/
!pip install -e .
!pip install einops

/content
Cloning into 'thermomaps-ising'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 235 (delta 126), reused 223 (delta 114), pack-reused 0
Receiving objects: 100% (235/235), 42.71 KiB | 1.64 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/content/thermomaps-ising/thermomaps-root
Obtaining file:///content/thermomaps-ising/thermomaps-root
  Preparing metadata (setup.py) ... done
  Running setup.py develop for thermomaps


In [2]:
import os
import ising
import numpy as np
from ising.swedsen_wang_simulator import IsingSwendsenWang, Energy, Magnetization
from tm.core.Backbone import ConvBackbone
from tm.core.DiffusionModel import DiffusionTrainer, SteeredDiffusionSampler
from tm.core.DiffusionProcesses import VPDiffusion
from tm.core.Loader import Loader
from tm.core.Prior import GlobalEquilibriumHarmonicPrior
from tm.architectures.UNet2D_mid_attn import Unet2D
from tm.core.utils import compute_model_dim

In [3]:
class IsingArgs:
  size = 8
  warm_up = 1000
  steps = 1000
  temp = 2
  Jx = 1
  Jy = 1
  sampling_frequency = 10
  filename = "ising"

In [ ]:
dataset, paired_temperatures, trajectory_fluctuation_dict = [], [], {}
temperatures = [1,2,3]

# Generating data at three temperatures using Swedsen-Wang sampling
for sim_idx, temp in enumerate(temperatures):

  observables = [Energy(Jx = IsingArgs.Jx, Jy = IsingArgs.Jy), Magnetization()]

  # Initialize simulation
  sim = IsingSwendsenWang(size = IsingArgs.size, warm_up = IsingArgs.warm_up,
                          temp = temp, Jx = 1, Jy = 1)

  # Run simulation
  results = sim.simulate(steps = IsingArgs.steps, observables = observables,
                         sampling_frequency = IsingArgs.sampling_frequency)

  # Coordinates are lattice configurations
  coordinates = np.expand_dims(results['lattice'], 1)

  # Fluctuations are temperatures (coordinates are sampled from the global equilibrium distribution)
  fluctuations = np.ones_like(coordinates) * temp

  # Form the TM input data by joining coordinates and fluctuations along the channel axis
  data = np.concatenate((coordinates, fluctuations), axis=1)

  dataset.append(data)

  # Each coordinate trajectory was collected at a temperature and has associated fluctuations
  # (include sim_idx in case there are repeat temperatures)
  trajectory_fluctuation_dict[f"{temp}_{sim_idx}"] = fluctuations

  # Each coordinate frame is paired with a temperature
  temps_arr = np.ones(len(coordinates)) * temp
  paired_temperatures.append(temps_arr)

dataset = np.concatenate(dataset)
paired_temperatures = np.concatenate(paired_temperatures)

The inputs to the `Loader` are the dataset and the temperatures associated with each sample, and control parameters. The control axis is the axis of the data containing the data that is to be controlled. For example, the default of `control_axis = 1` indicates that the control parameters may be retrieved along the channel dimension of the data, assuming the data is of shape `(n_samples, n_channels, x, y)`. In the provided example there are two channels: one containing lattice configurations and another containing the temperatures. Since the temperatures are the second dimension along the `control_axis`, we set `control_dims=(1,2)`. This way, the temperatures can be retrieved using a `loader.control_slice` via `loader.data[loader.control_slice]`.





In [ ]:
loader = Loader(data = dataset, temperatures = paired_temperatures,
                control_axis = 1, control_dims = (1,2))

The `GlobalEquilibriumHarmonicPrior` (`GEHP`) is responsible for sampling the prior at the correct temperature. The inputs to the `GEHP` is a dictionary `trajectory_fluctuation_dict` containing key-value pairs of generated coordinate trajectories, and the temperatures that the trajectories were generated at.

In [ ]:
GEHP = GlobalEquilibriumHarmonicPrior(data=trajectory_fluctuation_dict)

The format required as input for the `GEHP` might seem convoluted, but that is because there is also a `LocalEquilibriumHarmonicPrior` (`LEHP`) that performs the computations associated with fluctuation mapping. This involves computing fluctuations over each trajectory in the input dictionary, and fitting a multi-linear function to the (temperatures, fluctuation) data.

Next we initialize the black-box `model(nn.Module)` that will be parameterized during training. In this case, the `model` is a 2D U-net with attention at the bottleneck layer.

In [ ]:
# get model dim that is compatible w/ data shape + possible model shapes.
model_dim = compute_model_dim(loader.data_dim, groups=8)
model = Unet2D(
    dim=model_dim,
    dim_mults=(1, 2, 2, 4), # downsampling/upsampling factors before/after bottleneck
    resnet_block_groups=8,
    self_condition=True, # greatly improves training convergence when data is limited
    learned_sinusoidal_cond=True,
    channels=2, # two channels for Ising model data
)

The `backbone` is a wrapper class around `model` that contains methods used to load and save models, and automatically upsamples image-structured data to match the dimensions of the model.

In [ ]:
backbone = ConvBackbone(
    model=model,
    data_shape=loader.data_dim,
    target_shape=model_dim, # data is upsampled to this resolution
    num_dims=4, # (n_sample, n_channel, n_x, n_y)
    lr=1e-3,
    eval_mode="train",
    self_condition=True,
    )

The `VPDiffusion` class contains methods to evaluate forward and reverse diffusion kernels for the Variance Preserving diffusion process described in Song (2019).

In [ ]:
diffusion = VPDiffusion(num_diffusion_timesteps=100)

A `DiffusionTrainer` trains a `backbone` model to reverse the diffusion process (`VPDiffusion`) applied to coordinates (`dataset` in `Loader`). Optionally, the model can be trained to predict the original samples directly (`pred_type=x0`) or the noise used to corrupt the sample (`pred_type=noise`).

In [ ]:
trainer = DiffusionTrainer(diffusion, backbone, loader, model_dir='/content/TM/models',
                           pred_type='x0', prior=GEHP, device='cuda:0')
trainer.train(5, loss_type="l2", batch_size=16, print_freq=10)

Now sample from the thermodynamic map using the `SteeredDiffusionSampler`. First load the model corresponding to the epoch that you want to sample from, and then pass the `DiffusionProcess`, `Loader`, and `Prior` used to train the model. Optionally, provide a scalar `gamma` that conditions over the fluctuations during the generative process when `gamma=1`. Otherwise, if `gamma=0` the conditional process is carried out for the fluctuations dimensions.

In [ ]:
backbone.load_model('/content/TM/models', epoch=3, device='cuda:0')

sampler = SteeredDiffusionSampler(diffusion_process=diffusion, backbone=backbone,
                                  loader=loader, sample_dir='/content/TM/models/samples',
                                  pred_type='x0', prior=GEHP, gamma=1)

# gamma = 1 controls the diffusion of fluctuations based on the provided temperature.
# gamma = 0 allows the generative process to be carried out for the fluctuations

sampler.sample_loop(num_samples=1000, batch_size=1000, save_prefix='ising',
                    temperature=2.2, n_ch=2)
